In [14]:
import py_entitymatching as em
import csv
import os
import pandas as pd
import py_entitymatching as em

# reading cleaned csv files from stage 2
A = em.read_csv_metadata('./filmcrave1.csv', key='id')
B = em.read_csv_metadata('./imdb1.csv', key='id')

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


In [15]:
# Creating instance for overlap blocker.
ob = em.OverlapBlocker()

In [16]:
# Overlapping on Title. (Atleast 1 word match) - [Performed better than 3-gram]
overlap_candidate = ob.block_tables(A, B, 'Title', 'Title', word_level=True, overlap_size=1, rem_stop_words=False, 
                    l_output_attrs=['Title', 'Overall Rating', 'Year', 'Genre', 'Directors', 'Actors'], 
                    r_output_attrs=['Title', 'Overall Rating', 'Year', 'Genre', 'Directors', 'Actors'],
                    show_progress=False)

In [17]:
# Filtering previous stage blocker output further by matching atleast 1 word in directors
overlap_candidate1 = ob.block_candset(overlap_candidate, 'Directors', 'Directors', word_level=True, overlap_size=1, show_progress=False)

In [18]:
# Creating instance for attribute blocker
ab = em.AttrEquivalenceBlocker()

# Blocking previous level blocker using matching year attribute
attribute_candidate = ab.block_candset(overlap_candidate1, 'Year', 'Year', show_progress=False)

In [19]:
# Randomly sampling 500 records from the candidate tuple pairs
sample = em.sample_table(attribute_candidate, 500)

In [20]:
# Storing this sample for labelling.
sample.to_csv("Sampleset.csv")

In [27]:
# Opening the csv file and cleaning and converting to new CSV File
with open('./Sampleset.csv', 'r', encoding='utf-8', errors='ignore') as infile, open('./Sampleset1.csv', 'w') as outfile:
     inputs = csv.reader(infile)
     outputs = csv.writer(outfile)

     for index, row in enumerate(inputs):
         outputs.writerow(row)

# Read in the labelled dataset        
G = em.read_csv_metadata("./Sampleset1.csv", key='_id', 
                         fk_ltable='ltable_id', fk_rtable='rtable_id',
                         ltable=A, rtable=B)

Metadata file is not present in the given path; proceeding to read the csv file.


In [28]:
# Splitting the data into training and testing portions. 
split = em.split_train_test(G, train_proportion=0.5, random_state=0)

# Training and testing set splits
train_set = split['train']
test_set = split['test']

In [29]:
# Initialising all ML algos
dt = em.DTMatcher(name='DecisionTree', random_state=0)
svm = em.SVMMatcher(name='SVM', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')

In [30]:
# Generating features for training
features = em.get_features_for_matching(A, B, validate_inferred_attr_types=False)

In [31]:
# Extracting feature vectors to train and create model
H = em.extract_feature_vecs(train_set, 
                            feature_table=features, 
                            attrs_after='label',
                            show_progress=False)

In [32]:
H.head()

,_id,ltable_id,rtable_id,id_id_exm,id_id_anm,id_id_lev_dist,id_id_lev_sim,Title_Title_jac_qgm_3_qgm_3,Title_Title_cos_dlm_dc0_dlm_dc0,Title_Title_jac_dlm_dc0_dlm_dc0,...,Directors_Directors_nmw,Directors_Directors_sw,Actors_Actors_jac_qgm_3_qgm_3,Actors_Actors_cos_dlm_dc0_dlm_dc0,Actors_Actors_mel,Actors_Actors_lev_dist,Actors_Actors_lev_sim,Plot_Plot_jac_qgm_3_qgm_3,Plot_Plot_cos_dlm_dc0_dlm_dc0,label
476,1007330,3001,3298,0,0.909945,3,0.250000,0.259259,0.408248,0.250000,...,15.0,15.0,0.585714,0.166667,0.898730,17.0,0.730159,0.053892,0.069171,1
162,350997,1110,1147,0,0.967742,2,0.500000,0.450000,0.816497,0.666667,...,12.0,12.0,0.537313,0.166667,0.871178,19.0,0.666667,0.216667,0.193601,1
34,63043,39,202,0,0.193069,3,0.000000,0.230769,0.408248,0.250000,...,12.0,12.0,0.557377,0.182574,0.881275,15.0,0.705882,0.124000,0.141069,1
44,85897,243,280,0,0.867857,2,0.333333,0.153846,0.707107,0.500000,...,12.0,12.0,0.558824,0.182574,0.882112,17.0,0.711864,0.216138,0.243778,1
97,168032,847,543,0,0.641086,2,0.333333,1.000000,1.000000,1.000000,...,13.0,13.0,0.346154,0.182574,0.667645,36.0,0.389831,0.321192,0.358209,1


In [33]:
any(pd.notnull(H))

True

In [34]:
H = em.impute_table(H, 
                exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'label'],
                strategy='mean')

In [35]:
result = em.select_matcher([dt, rf, svm, ln, lg], table=H, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'label'],
        k=5,
        target_attr='label', metric_to_select_matcher='f1', random_state=0)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/linalg/basic.py:1226: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/linalg/basic.py:1226: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/linalg/basic.py:1226: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.

In [36]:
result['cv_stats']

,Matcher,Average precision,Average recall,Average f1
0,DecisionTree,0.989717,0.995122,0.992322
1,RF,1.000000,0.995122,0.997531
2,SVM,0.895191,1.000000,0.943919
3,LinReg,1.000000,1.000000,1.000000
4,LogReg,0.994595,0.995122,0.994791


In [37]:
dt = em.DTMatcher(name='LinReg', random_state=0)

In [38]:
dt.fit(table=H, 
       exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'label'], 
       target_attr='label')

In [40]:

L = em.extract_feature_vecs(test_set, feature_table=features,
                            attrs_after='label', show_progress=False)

In [41]:
any(pd.notnull(L))

True

In [42]:
L = em.impute_table(L, 
                exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'label'],
                strategy='mean')

In [43]:
predictions = dt.predict(table=L, exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'label'], 
              append=True, target_attr='predicted', inplace=False, return_probs=True,
                        probs_attr='proba')

In [35]:
predictions[['_id', 'ltable_id', 'rtable_id', 'predicted', 'proba']].head()

,_id,ltable_id,rtable_id,predicted,proba
90,155959,163,499,1,1.0
254,529437,1959,1736,1,1.0
283,603984,2180,1972,1,1.0
445,936841,3176,3076,1,1.0
461,971241,3124,3178,1,1.0


In [44]:
eval_result = em.eval_matches(predictions, 'label', 'predicted')
em.print_eval_summary(eval_result)

Precision : 97.33% (182/187)
Recall : 98.38% (182/185)
F1 : 97.85%
False positives : 5 (out of 187 positive predictions)
False negatives : 3 (out of 63 negative predictions)
